In [ ]:
#Import required libraries
import requests
import pandas as pd
import sys

In [ ]:
#Create a function to flatten nested dictionaries or lists from a json
def flatten_json(nested_json, exclude=['']):
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [ ]:
#Create a function to request data to a URL and handle errors
def getdata(url):
  try:
    r = requests.get(url)
    r.raise_for_status()
    return requests.get(url).json()
  except requests.exceptions.HTTPError as e:
    print (e.response.text)
    sys.exit(1)

In [ ]:
#List of key words I want to search in Mercado Libre's API
items = ['chromecast', 'apple_tv', 'Amazon_Fire', 'roku']
index = 0

#Loop through the list of keywords to get data for each keyword and append that data to a Pandas dataframe
for item in items:
  url = "https://api.mercadolibre.com/sites/MLA/search?q={0}&limit=50#options".format(item)
  rawdata = getdata(url)
  if index == 0:
    df = pd.DataFrame([flatten_json(x) for x in rawdata['results']])
    df['searched'] = item
  else:
    df2 = pd.DataFrame([flatten_json(x) for x in rawdata['results']])
    df2['searched'] = item
    df = pd.concat([df, df2], ignore_index=True)

  index += 1

In [ ]:
#Setting a goal to clean the Pandas dataframe and only keep columns that have more than 20% of its rows populated
mingoal = len(df.index) * 0.20

40.0

In [ ]:
#Clean the dataframe using the goal
for col in df.columns:
  if df[col].notnull().sum() <= mingoal:
    df.drop(col,inplace=True,axis=1)
  
# print(df[col].notnull().sum(), col)

In [ ]:
#Create a .csv file from the dataframe
df.to_csv('ML_data.csv')